### * Task  주어진 json형태로, Input 문장이  Output 카테고리 분류 
* Input : text, Output : parentId
* parentId가 null값이 아닌것들로만 추출
* 0.5초 이하 검색시간

In [ ]:
import json
import pandas as pd
with open ("category.json", "r") as f:
        data = json.load(f)
df = pd.DataFrame(data)

In [ ]:
# 카테고리에 음악이 중복이 있어 혼동을 줄수 있음으로 교육이라는 값을 뒤에 붙임
def mrg(df):
   
    # parentId null 값 제거
    df = df[df['parentId'].notnull()]
    df['parentId'].value_counts()
    # 필요 컬럼만 남기기
    df = df[['id',	'parentId',	'title', 'pathname']]
    df

    # 한글로 기대된 parentId(대분류) 만들기 < 전 task의 데이터셋에서 카테고리명참고, 네이버지식인 카테고리참고
    parentId_dict = { 'parentId' : [1.0, 18.0, 31.0, 41.0, 57.0, 63.0, 68.0, 74.0, 78.0, 83.0, 93.0, 99.0, 103.0, 117.0, 136.0, 156.0, 176.0, 184.0, 198.0, 212.0, 226.0, 235.0, 241.0, 247.0] ,
                    'KoparentId': ['법률', '세금세무', '고용노동', '건강상담', '건강관리', '약영양제', '심리', '반려동물', '육아', '경제', '보험', '제품수리', '학문', '취업', '입시', '자격증', '고민상담', '디지털가전제품', '스포츠운동', '여행', '취미여가활동', '방송미디어', '생활', 'AMA']}
    # 기존 df + PI_df join
    pI_df = pd.DataFrame(parentId_dict)
    mrg_df = pd.merge(pI_df,df)
    return mrg_df

In [ ]:
# json 데이터를 불러와서 대분류(KoparentId) 중분류(title)이 있는 데이터 프레임으로 만들기
mrg_df = mrg(df)

In [ ]:
#ver1 작업
# # sbert가 문장 두개를 비교하는 model임
# # 카테고리 단어 > 문장으로 변환
# mrg_df['re_title'] = mrg_df['title'] + mrg_df['KoparentId'] + "주제에 관련하여 문의드립니다."

In [ ]:
# 중복값없음 label로 사용될 카테고리 개수
len(set(df['title'].values.tolist()))
len(df['title'].values.tolist())

* zeroshot
* 세가지 사용해봄 mDeBERTav3basemnlixnli / roberta_with_kornli / bartlargemnli 

In [ ]:
# zeroshotclassification mDeBERTa
from transformers import pipeline
classifier = pipeline("zeroshotclassification", model="MoritzLaurer/mDeBERTav3basemnlixnli")

import time
start_time = time.time()
sequence_to_classify = '배가 아파요'
category_label = df['title'].values.tolist()
output = classifier(sequence_to_classify, category_label, multi_label=False)
print(f"입력 문장: {sequence_to_classify}")
print("KoparentId :", mrg_df[mrg_df['title'] == output['labels'][output['scores'].index(max(output['scores']))]]['KoparentId'].values[0])
print("parentId :", mrg_df[mrg_df['title'] == output['labels'][output['scores'].index(max(output['scores']))]]['parentId'].values[0])
print('title :', output['labels'][output['scores'].index(max(output['scores']))])
print("확률 :",round(output['scores'][output['scores'].index(max(output['scores']))], 4))

end_time = time.time()
search_time = end_time - start_time
print(f"검색 시간: {search_time:.4f} 초")

In [ ]:
# pongjin/roberta_with_kornli
import time


classifier = pipeline(
    "zeroshotclassification",
    model="pongjin/roberta_with_kornli"
)
start_time = time.time()
sequence = "배가 아파요"	
category_label = df['title'].values.tolist()

output = classifier(
    sequence,
    category_label,
)

print(f"입력 문장: {sequence}")
print("KoparentId :", mrg_df[mrg_df['title'] == output['labels'][output['scores'].index(max(output['scores']))]]['KoparentId'].values[0])
print("parentId :", mrg_df[mrg_df['title'] == output['labels'][output['scores'].index(max(output['scores']))]]['parentId'].values[0])
print('title :', output['labels'][output['scores'].index(max(output['scores']))])
print("확률 :",round(output['scores'][output['scores'].index(max(output['scores']))], 4))

end_time = time.time()

search_time = end_time - start_time
print(f"검색 시간: {search_time:.4f} 초")

------------------------------------------------

* Topic classify ver2 - 추가작업 (10/10일)

* 업무 프로세스
    1. 각종 전문 사이트 및 연관검색어를 활용하여 소분류에 해당하는 데이터 수집

    1. 수집한 데이터를 한글과 띄워쓰기의 일정한 규격에 맞게 전처리

    1. Annoy index에 저장
        1. (angular, euclidean, manhattan, hamming, dot) 거리 방법 마다 .ann 파일로 저장

    1. sbert model 6가지를 각각 실행
    
    1. 특정 문장 기준, sbert model과 각각의 거리 지표화 (5*6)의 경우의수

### - 데이터 수집

In [ ]:
# 기재가 안되어있는 주제 참고 사이트 : 구글에서 연관검색어, 관련 블로그 참고, 썸트렌드 연관검색어
# 법률  https://www.easylaw.go.kr/(찾기쉬운 생활법령정보), google 트렌드
# 의료   자생한방병원, 차병원, 서울대학교 병원, 공공데이터 포탈 : 한국보훈복지의료공단_외래 진료과별 상위20 주요상병_대전보훈병원
# 자격증  google 트렌드
# 학교, 기업, 여행/지역  구글 검색(관련 검색어)
# 가전  https://www.domero.net(B2B 위탁판매 플렛폼 도매로), https://category.gmarket.co.kr/(G마켓)
# 교육/학문  https://ko.wikibooks.org/(주제별 위키책), https://humanities.snu.ac.kr/(서울대학교 인문대학 커리큘럼기준)
# 세금  https://www.nts.go.kr(국세청), http://www.eanse.com/page/page22(회계법인 사이트)
# 금융  https://ko.wikipedia.org/(위키피디아), https://banksalad.com(뱅크샐러드), https://www.dongacc.com/online/textbook/0668_report.pdf(라임사이버 문화센터)
# 취미  http://snuaaa.com/(산악회사이트용어 수집), https://zanca.co.kr/article/%EC%9E%94%EC%B9%B4%EC%8A%A4%ED%86%A0%EB%A6%AC/12/1911/(낚시 용어), https://ymavenue.com/%EC%BA%A0%ED%95%91%EC%9A%A9%EC%96%B4%EC%A0%95%EB%A6%AC1/(캠핑용어 정리), https://www.adobe.com/kr/creativecloud/photography/discover/typesofphotography.html(사진용어 정리)
# 상담  관련 블로그 여럿 참조.
# 웹소설 웹툰 https://ridibooks.com/category/list(리디)

detail_title = {
    '가족·이혼' :  '가족관계등록 가족관계등록부 특정등록사항 일반등록사항 등록기준지 호적 등록사항별증명서 증명서발급 출생신고 인지신고 입양신고 미성년자입양 친양자입양 파양 재판상 파양 협의상 파양 친양자파양 친양자파양효력 입양취소 혼인신고 혼인취소'
    ,'형사' : '의료분쟁 의료사고 의료행위 보건의료인 합의 조정 중재 소송 손해배상청구소송 형사고소 고발 진료기록부 한국의료분쟁조정중재원 소비자상담센터 한국소비자원 대한법률구조공단 대한변협법률구조재단 피해구제 형사소송 행정소송 가사소송'
    ,'폭행·협박' : '학교폭력 피해학생 가해학생 학교폭력 유형 학교폭력 징후 전문상담교사 미성년자 폭행 법정대리인 학교 상담실 전담기구 학생보호인력 학교전담경찰관 학교폭력 예방교육 신고 고발 117 비밀누설금지 학교폭력 조치사항 사안조사 사안보고 자체해결 학교장 학교폭력대책심의위원회'
    ,'명예훼손·모욕' : '온라인명예훼손 인터넷명예훼손 전산망명예훼손 사이버권리침해 사이버명예훼손 분쟁조정 임시조치 인터넷 가해자 정보 온라인범죄 사이버범죄 비방 인터넷 개인방송비방 유튜브비방 인터넷 방송비방 융합미디어 방송 심의 통신 심의 방송통신심의위원회 플랫폼'
    ,'재산범죄' : '고소 고발 수사 형사조정 공판 화해 배상 소액사건심판 지급명령 민사조정 민사소송 형집행 보호관찰 상담 보호시설 치료비 심리치료비 생계비 학자금 장례비 국민임대주택'
    ,'교통사고' : '출퇴근길 대중교통 이용 출퇴근 버스 출퇴근 지하철 출퇴근 부정승차 출퇴근길 안전 교통약자의 이동권 보장 지하철 안전관리 버스 안전관리 버스폭발 사고 버스기사 폭행 처벌 대중교통 범죄 예방 및 대처방법 유실물 찾기 출퇴근길 교통사고 출퇴근길 사고발생 대처방법 출퇴근 자전거 운전자 교통법규 자전거 주차장 무단방치자전거 자동차 출퇴근 오토바이 출퇴근'
    ,'성범죄' : '성범죄 성폭력 성매매 신고 상담 고소 공소시효 손해배상 신원 사생활 보호시설 지원시설 법률지원 긴급지원 신상정보 전자장치 약물치료'
    ,'민사' : '민사소송 행정소송 가사소송 비송사건 구분 간이구제절차 민사조정 화해 지급명령 공시최고 제권판결 소액심판 이행권고 나홀로민사소송 송달료 전자소송 채권추심 소액사건재판 법률구조제도'
    ,'부동산·임대차' : '상가 임대차 확정일자 전세권 전전세 임대인 임차인 상가건물 대항력 보증금 우선변제권 소액임차인 부동산등기부 부동산중개업자 임대차계약서 차임 갱신 합의갱신 묵시의 갱신 임차권등기명령 집행권원 건폐율 용적률'
    ,'가압류·가처분' : '공탁 변제공탁 형사변제공탁 담보공탁 보증공탁 집행공탁 보관공탁 몰취공탁 혼합공탁 공탁대상 공탁물 공탁금 당사자능력 미성년자 법정대리인 행위능력 공탁행위 피한정후견인 피성년후견인 공탁소 공탁관'
    ,'회생·파산' : '개인회생 파산 개인회생 신청자격 개인회생절차 신청서 신청서 제출 채권자목록 재산목록 수입 및 지출에 관한 목록 진술서 금지 및 중지명령 신청서 변제계획안 인지대 송달료 공고비용 개인회생 개시결정  개시결정문  이의제기  파산 절차 중지  가압류ㆍ가처분 중지  변제 중지'
    ,'금융' : '일반금융소비자 금융소비자보호법 금융상품 6대판매원칙 적합성원칙 적정성원칙 부당권유금지 설명의무 손해배상책임 청약철회권 위법계약해지권 자료열람요구권 금리인하요구권 안내의무 휴면예금 예금자보호 예금자보호법 금융분쟁조정 펀드 펀드투자 펀드투자자 금융투자'
    ,'의료' : '의료분쟁 의료사고 의료행위 보건의료인 합의 조정 중재 소송 손해배상청구소송 형사고소·고발 진료기록부 한국의료분쟁조정중재원 소비자상담센터 한국소비자원 대한법률구조공단 대한변협법률구조재단 피해구제 연명 안락사 존엄사 웰다잉'
    ,'기업·회사' : '가장납입 감정인 검사인 공증 공증인 기업지원플러스센터 납입가장죄 납입금 상계 등기신청인 등록면허세 명의개서 대리인 모집설립 무액면주식 발기설립 발기인 조합 법원의 해산명령 법인 법인성 변태설립사항 보통주 사업자등록'
    ,'기타 법률상담' : "환경법 이민법 국제법 건강법 의료 법 환경법"
    ,'지식재산권·IT' : '특허권 발명 선출원 특허요건 특허증 특허출원 특허등록 특허료 연차등록료 등록보정 특허기간 산업재산권 지식재산권 전용권 전용실시권 통상실시권 선행기술조사 변리 특허비용 특허상담 특허청'
    ,'변호사 자격증' : '변호사 자격증 시험일정 변호사 자격증 시험과목 변호사 자격증 난이도 변호사 자격증 합격률'
    ,'변리사 자격증' : '변리사 자격증 시험일정 변리사 자격증 시험과목 변리사 자격증 난이도 변리사 자격증 합격률'
    ,'공인회계사 자격증' : 'cpa cpa시험 공인회계사 자격증 시험일정 공인회계사 자격증 시험과목 공인회계사 자격증 난이도 공인회계사 자격증 합격률'
    ,'세무사 자격증' : '세무사 자격증 시험일정 세무사 자격증 시험과목 세무사 자격증 난이도 세무사 자격증 합격률'
    ,'감정평가사 자격증' : '감정평가사 자격증 시험일정 감정평가사 자격증 시험과목 감정평가사 자격증 난이도 감정평가사 자격증 합격률'
    ,'손해사정사 자격증' : '손해사정사 자격증 시험일정 손해사정사 자격증 시험과목 손해사정사 자격증 난이도 손해사정사 자격증 합격률'
    ,'관세사 자격증' : '관세사 자격증 시험일정 관세사 자격증 시험과목 관세사 자격증 난이도 관세사 자격증 합격률'
    ,'공인노무사 자격증' : '공인노무사 자격증 시험일정 공인노무사 자격증 시험과목 공인노무사 자격증 난이도 공인노무사 자격증 합격률'
    ,'법무사 자격증' : '법무사 자격증 시험일정 법무사 자격증 시험과목 법무사 자격증 난이도 법무사 자격증 합격률'
    ,'경영지도사 자격증' : '경영지도사 자격증 시험일정 경영지도사 자격증 시험과목 경영지도사 자격증 난이도 경영지도사 자격증 합격률'
    ,'간호조무사 자격증' : '간호조무사 자격증 시험일정 간호조무사 자격증 시험과목 간호조무사 자격증 난이도 간호조무사 자격증 합격률'
    ,'사회복지사 자격증' : '사회복지사 자격증 시험일정 사회복지사 자격증 시험과목 사회복지사 자격증 난이도 사회복지사 자격증 합격률'
    ,'행정사 자격증' : '행정사 자격증 시험일정 행정사 자격증 시험과목 행정사 자격증 난이도 행정사 자격증 합격률'
    ,'정보처리기사' : '정보처리기사 시험일정 정보처리기사 시험과목 정보처리기사 난이도 정보처리기사 합격률'
    ,'산업안전산업기사' : '산업안전산업기사 시험일정 산업안전산업기사 시험과목 산업안전산업기사 난이도 산업안전산업기사 합격률'
    ,'전기기사·기능사' : '전기기사·기능사 시험일정 전기기사·기능사 시험과목 전기기사·기능사 난이도 전기기사·기능사 합격률'
    ,'지게차운전기능사' : '지게차운전기능사 시험일정 지게차운전기능사 시험과목 지게차운전기능사 난이도 지게차운전기능사 합격률'
    ,'보험설계사 자격증' : '보험설계사 자격증 시험일정 보험설계사 자격증 시험과목 보험설계사 자격증 난이도 보험설계사 자격증 합격률'
    ,'공인중개사 자격증' : '공인중개사 자격증 시험일정 공인중개사 자격증 시험과목 공인중개사 자격증 난이도 공인중개사 자격증 합격률'
    ,'서울대학교' : '서울대학교 수시 서울대학교 정시 서울대학교 합격률 서울대학교 취업률 서울대학교 학과 서울대학교 축제 서울대학교 캠퍼스'
    ,'연세대학교' : '연세대학교 수시 연세대학교 정시 연세대학교 합격률 연세대학교 취업률 연세대학교 학과 연세대학교 축제 연세대학교 캠퍼스'
    ,'고려대학교' : '고려대학교 수시 고려대학교 정시 고려대학교 합격률 고려대학교 취업률 고려대학교 학과 고려대학교 축제 고려대학교 캠퍼스'
    ,'포항공과대학교' : '포항공과대학교 수시 포항공과대학교 정시 포항공과대학교 합격률 포항공과대학교 취업률 포항공과대학교 학과 포항공과대학교 축제 포항공과대학교 캠퍼스'
    ,'성균관대학교' : '성균관대학교 수시 성균관대학교 정시 성균관대학교 합격률 성균관대학교 취업률 성균관대학교 학과 성균관대학교 축제 성균관대학교 캠퍼스'
    ,'한양대학교' : '한양대학교 수시 한양대학교 정시 한양대학교 합격률 한양대학교 취업률 한양대학교 학과 한양대학교 축제 한양대학교 캠퍼스'
    ,'이화여자대학교' : '이화여자대학교 수시 이화여자대학교 정시 이화여자대학교 합격률 이화여자대학교 취업률 이화여자대학교 학과 이화여자대학교 축제 이화여자대학교 캠퍼스'
    ,'서강대학교' : '서강대학교 수시 서강대학교 정시 서강대학교 합격률 서강대학교 취업률 서강대학교 학과 서강대학교 축제 서강대학교 캠퍼스'
    ,'UNIST' : 'UNIST 수시 UNIST 정시 UNIST 합격률 UNIST 취업률 UNIST 학과 UNIST 축제 UNIST 캠퍼스'
    ,'GIST' : 'GIST 수시 GIST 정시 GIST 합격률 GIST 취업률 GIST 학과 GIST 축제 GIST 캠퍼스'
    ,'경희대학교' : '경희대학교 수시 경희대학교 정시 경희대학교 합격률 경희대학교 취업률 경희대학교 학과 경희대학교 축제 경희대학교 캠퍼스'
    ,'중앙대학교' : '중앙대학교 수시 중앙대학교 정시 중앙대학교 합격률 중앙대학교 취업률 중앙대학교 학과 중앙대학교 축제 중앙대학교 캠퍼스'
    ,'의과대학' : '의과대학 수시 의과대학 정시 의과대학 합격률 의과대학 취업률 의과대학 학과 의과대학 축제 의과대학 캠퍼스'
    ,'치과대학' : '치과대학 수시 치과대학 정시 치과대학 합격률 치과대학 취업률 치과대학 학과 치과대학 축제 치과대학 캠퍼스'
    ,'로스쿨' : '로스쿨 수시 로스쿨 정시 로스쿨 합격률 로스쿨 취업률 로스쿨 학과 로스쿨 축제 로스쿨 캠퍼스'
    ,'한의대학' : '한의대학 수시 한의대학 정시 한의대학 합격률 한의대학 취업률 한의대학 학과 한의대학 축제 한의대학 캠퍼스'
    ,'KAIST' : 'KAIST 수시 KAIST 정시 KAIST 합격률 KAIST 취업률 KAIST 학과 KAIST 축제 KAIST 캠퍼스'
    ,'서울시립대학교' : '서울시립대학교 수시 서울시립대학교 정시 서울시립대학교 합격률 서울시립대학교 취업률 서울시립대학교 학과 서울시립대학교 축제 서울시립대학교 캠퍼스'
    ,'한국외국어대학교' : '한국외국어대학교 수시 한국외국어대학교 정시 한국외국어대학교 합격률 한국외국어대학교 취업률 한국외국어대학교 학과 한국외국어대학교 축제 한국외국어대학교 캠퍼스'
    ,'삼성' : '삼성 계열사 삼성 재무재표 삼성 매출액 삼성 평점 삼성 채용 삼성 연봉 삼성 복지 삼성 자소서 이력서'
    ,'SK' : 'SK 계열사 SK 재무재표 SK 매출액 SK 평점 SK 채용 SK 연봉 SK 복지 SK 자소서 이력서'
    ,'LG' : 'LG 계열사 LG 재무재표 LG 매출액 LG 평점 LG 채용 LG 연봉 LG 복지 LG 자소서 이력서'
    ,'롯데' : '롯데 계열사 롯데 재무재표 롯데 매출액 롯데 평점 롯데 채용 롯데 연봉 롯데 복지 롯데 자소서 이력서'
    ,'포스코' : '포스코 계열사 포스코 재무재표 포스코 매출액 포스코 평점 포스코 채용 포스코 연봉 포스코 복지 포스코 자소서 이력서'
    ,'한화' : '한화 계열사 한화 재무재표 한화 매출액 한화 평점 한화 채용 한화 연봉 한화 복지 한화 자소서 이력서'
    ,'GS' : 'GS 계열사 GS 재무재표 GS 매출액 GS 평점 GS 채용 GS 연봉 GS 복지 GS 자소서 이력서'
    ,'현대중공업' : '현대중공업 계열사 현대중공업 재무재표 현대중공업 매출액 현대중공업 평점 현대중공업 채용 현대중공업 연봉 현대중공업 복지 현대중공업 자소서 이력서'
    ,'신세계' : '신세계 계열사 신세계 재무재표 신세계 매출액 신세계 평점 신세계 채용 신세계 연봉 신세계 복지 신세계 자소서 이력서'
    ,'KT' : 'KT 계열사 KT 재무재표 KT 매출액 KT 평점 KT 채용 KT 연봉 KT 복지 KT 자소서 이력서'
    ,'CJ' : 'CJ 계열사 CJ 재무재표 CJ 매출액 CJ 평점 CJ 채용 CJ 연봉 CJ 복지 CJ 자소서 이력서'
    ,'카카오' : '카카오 계열사 카카오 재무재표 카카오 매출액 카카오 평점 카카오 채용 카카오 연봉 카카오 복지 카카오 자소서 이력서'
    ,'네이버' : '네이버 계열사 네이버 재무재표 네이버 매출액 네이버 평점 네이버 채용 네이버 연봉 네이버 복지 네이버 자소서 이력서'
    ,'토스' : '토스 계열사 토스 재무재표 토스 매출액 토스 평점 토스 채용 토스 연봉 토스 복지 토스 자소서 이력서'
    ,'구글' : '구글 계열사 구글 재무재표 구글 매출액 구글 평점 구글 채용 구글 연봉 구글 복지 구글 자소서 이력서'
    ,'LINE' : 'LINE 계열사 LINE 재무재표 LINE 매출액 LINE 평점 LINE 채용 LINE 연봉 LINE 복지 LINE 자소서 이력서'
    ,'배달의민족' : '배달의민족 계열사 배달의민족 재무재표 배달의민족 매출액 배달의민족 평점 배달의민족 채용 배달의민족 연봉 배달의민족 복지 배달의민족 자소서 이력서'
    ,'현대자동차' : '현대자동차 계열사 현대자동차 재무재표 현대자동차 매출액 현대자동차 평점 현대자동차 채용 현대자동차 연봉 현대자동차 복지 현대자동차 자소서 이력서'
    ,'내과' : '본태성 일차성 고혈압인슐린비의존 당뇨병만성 콩팥 신장 기능상실지질단백질대사장애기타 지질증부종수분. 전해질 및 산염기균형의 기타장애위염 및 십이지장염협심증상세불명의 당뇨병고립성 단백뇨증위식도역류병U07의 응급사용기타 만성 폐쇄성 폐질환인슐린의존 당뇨병기타 갑상선기능저하증비뇨기계통의 기타 장애영양실조와 관련된 당뇨병'
    ,'이비인후과' : '이물감 수면무호흡증 코기능성형 축농증내시경수술 알레르기비염 구강암 설암 구인두암 편도암 갑상선암 인공와우이식 중이염 유착성중이염 돌발성난청 난청(청력소실) 이명(귀울림) 보청기 비부비동종양 두개저내시경수술 비중격만곡증 코골이 수면무호흡 만성부비동염'
    ,'안과' : '녹내장안검의 기타 염증노인성 백내장눈물기관의 장애기타 백내장달리 분류된 질환에서의 망막장애기타 망막장애안검의 기타 장애각막염망막혈관폐쇄결막의 기타 장애결막염초자체의 장애굴절 및 조절의 장애시각장애기타 기능적 삽입물의 존재망막박리 및 망막이단 각막의 기타 장애마비성 사시맥립종 및 콩다래끼'
    ,'소아청소년과' : '천식 아토피피부염 신생아학 신생아 비만 중이염 소아내분비 소아신경 미숙아 비염 갑상선질환 두드러기 영양 갑상선 알레르기 질환 성장 미숙아성장 사춘기장애 당뇨 성조숙증 미숙아및신생아질환 성장및발달 육아상담 소아신경학 신생아질환 및 성장발달 소아심장 소아감염 및 면역질환 공단검진 터너증후군 육아지도 식품 알레르기 신생아 귀교정 신생아 성장과 발달 성장과 발달 미숙아 질환 소아성장 성장호르몬 프래더윌리증후군'
    ,'피부과' : '소양증피부사상균증앨러지성 접촉피부염지루피부염만성 단순태선 및 양진대상포진기타 피부염티눈 및 굳은살헤르페스바이러스 단순헤르페스 감염지루각화증기타 모낭장애두드러기건선달리 분류된 질환에서의 뇌신경장애질병이환 및 사망의 외인위염 및 십이지장염안드로젠탈모증옴바이러스성 사마귀여드름'
    ,'성형외과' : '코성형 유방성형 유방재건 눈성형 주름 성형술 일반 미용성형'
    ,'비뇨의학과' : '전립선비대증 방광암 신장암 전립선암 요로결석 배뇨장애 전립선질환 남성난임 무정자증 정계정맥류 여성요실금 비뇨기종양 내시경수술 남성가임력보존 일반비뇨기질환 폐쇄성 무정자증 태아 신생아 수신증 요도하열 왜소음경 잠복고환 음낭수종/탈장 방광요관역류 소아배뇨장애'
    ,'산부인과' : '자궁선근증 자궁경부암 로봇수술 자궁내막증 태아기형진단 부인과학 부인종양학 부인종양 고위험임신 자궁경수술 복강경수술 갱년기 다태임신 난소암 골반장기탈출증 자궁내막암 쌍태임신 요실금 자궁근종 부인암 국제진료 초음파 단일공 복강경수술 폐경기 검진 비정상자궁경부세포진 자궁내시경수술 난소종양 골반내시경수술 비뇨부인과 자궁난소 종양 자궁 질탈출 성형 여성갱년기 진단과 치료 골반저부질환(자궁탈증, 질탈증, 방광탈증) 외음부 및 질성형수술 산과 부인과 임신중산전관리 및 검사 성교불쾌증 성교통 불감증 질경련증 일반부인과질환 일반산부인과 단일공수술 산전관리 생리불순 일반 부인과 월경이상 정밀초음파 태아 기형 고위험 임신 임신중 약물상담 임신 전 상담 자연주의 출산 임신 중 검사 및 관리 일반부인과 자궁내막 폴립 점막하 자궁근종 임신 중 산전관리 단일공을 비롯한 복강경 수술 다빈치 로봇수술 부인암 진료 유전상담 태아발육부전 자궁내태아치료 자궁탈출증 난소양성종양 비정상 자궁경부 세포진 복강경 수술 임신전 상담 태아정밀진단 태아치료 쌍둥이임신 다낭난소증후군 미성년클리닉 폐경기 클리닉 골반장기재건술 산부인과레이저 클리닉 부인과 종양학(자궁경부암, 난소암, HPV) 양성자궁종양 복강경 부인암 수술 복강경 부인종양 수술'
    ,'정형외과' : '척추병증 윤충증 어깨병변 진균증추간판장애 변형성 배병증달리 분류되지 않은 처치후 근육골격 장애병적골절이 없는 골다공증허리뼈 및 골반의 골절늑골. 흉골 및 흉추골의 골절유독성 물질에 의한 불의의 중독 및 노출대퇴골의 골절배통아래팔의 골절말초혈관질환달리 분류되지 않은 기타 연조직장애발목을 포함한 아래다리의 골절달리 분류되지 않은 기타 관절장애활막염 및 건초염'
    ,'신경외과' : '척추병증 배통 대뇌혈관 질환머리내 손상대뇌혈관 질환의 후유증뇌내출혈추간판장애머리의 손상의 후유증뇌경색증척추증목뼈원판 장애달리 분류되지 않은 처치후 근육골격 장애대뇌경색증을 유발하지 않은 뇌전동맥의 폐색 및 협착두통목부위의 관절 및 인대의 탈구, 염좌 및 긴장허리뼈 및 골반의 관절 및 인대의 탈구, 염좌 및 긴장뇌 및 중추신경계통의 행동양식 불명 또는 미상의 신생물머리의 표재성 손상U07의 응급사용달리 분류되지 않은 피부 및 피하조직의 기타 장애'
    ,'치과' : '구강악안면외과 턱 교정 소아청소년교정 양악수술 주걱턱 돌출입 긴얼굴 안면비대칭 무턱 보철과 보존 교정과 치과 교정학 성인교정 심미교정 수술교정'
    ,'치아 관리' : '구강악안면외과 턱 교정 소아청소년교정 양악수술 주걱턱 돌출입 긴얼굴 안면비대칭 무턱 보철과 보존 교정과 치과 교정학 성인교정 심미교정 수술교정'
    ,'한의학' : '허리디스크 퇴행성디스크 척추수술후유증 척추관협착증 척추방전위증 척추분리증 스포츠상해 만성요통목디스크 경추신경증 일자목 두통 목 어깨 팔 등의 근골격계 질환턱관절디스크 턱관절기능성장애(안면비대칭 음식섭취장애 두통 견비통) 목디스크퇴행성관절염 류머티스관절염 퇴행성척추관절질환 척추압박골절 골다공증 오십견 스포츠상해성장 만성피로 여성 산전산후 비염 축농증 아토피 보약MRI CT XRAY 초음파 골밀도 혈액검사 등을 이용한 척추디스크 등 각종 질환 진단'
    ,'재활·물리치료' : '경부통 슬관절통 근골격계 및 스포츠 재활 요통 견관절통 암재활 부종 암통증 손발저림 암환자피로 암환자 쇠약 뇌질환재활 외상성뇌손상 뇌졸중 삼킴장애 척추 근골격계재활 신경근육질환 근전도 수술중신경생리 감시 연하장애 뇌질환재활 파킨슨병 뇌졸중 기타 뇌질환 연하장애 성인 뇌성마비 근골격계재활 절단재활 심장재활 호흡재활 섭식연하재활 중환자재활 추 근골격계재활'
    ,'기타 의료상담' : "정신의학과 노화 약물"
    ,'서울·수도권' : '서울 근교 바람쐬기 좋은 곳 서울 근교 당일치기 여행 서울근교 힐링여행 서울 근교 뚜벅이여행 서울 근교 당일 드라이브 차없이 당일치기 여행 서울 근교 대중교통 여행 사람 없는 서울 근교'
    ,'강원' : '강원도 가볼만한곳 베스트10 강원도 1박2일 여행코스 강원도 여행코스 강원도 여행지 순위강원도 예쁜 곳 강원도 바다 여행 강원도 유명한 곳 강원도 관광'
    ,'충청' : '충청도 1박2일 여행 충남 충북 가볼만한 곳 충청도 숨은 명소 충청도 2박 3일 여행 충청도 단양 여행 충청북도 여행 충청관광'
    ,'경상' : '경상도 여행코스 경상도 혼자 여행 경상도 1박2일 여행지 경상도 2박3일 여행지 경상북도 여행코스 경상도 데이트코스 경상도 숨은 명소 경상도 뚜벅이여행'
    ,'부산' : '부산 여행 코스 정리 1박2일 부산여행 코스 정리 부산 여행 코스 2박3일 부산 특이한 곳 부산여행 부산 20대 놀거리 부산 여행 필수 코스 부산 즐길거리'
    ,'전라' : '남도여행 코스 전남 여행코스 전남 1박2일 여행 남도 여행 추천 전라도 가볼만한 곳 전남 숨은 명소 전라도 겨울 여행 전라북도 여행'
    ,'제주' : '제주도 여행지 추천 제주도 여행코스 3박4일 제주도 여행 코스 2박 3일 20대 제주도 여행 제주도 유명한 곳제주도 여행 비용 제주도 여행 코스 4박 5일 제주도 관광지 순위'
    ,'일본' : '일본 관광지 순위 한국인 없는 일본 여행지 한국인이 좋아하는 일본 관광지 일본 특이한 여행지 일본 여행 코스 일본 관광지 베스트 일본 지방 여행첫 일본 여행 추천'
    ,'중국' : '죽기 전에 가봐야 할 중국 여행지 중국 여행 코스 중국 여행 가능한가요 중국 역사관광지 중국 10대 관광지 중국 관광명소 중국 여행 후기 중국여행 패키지'
    ,'동남아' : '동남아 여행 패키지 동남아 여행 코스 동남아 여행지 순위 동남아 여행 가능 국가 동남아 휴양지 추천 안전한 동남아 여행지 동남아 여행지 추천 동남아 여행지 추천'
    ,'유럽·아프리카' : '유럽 패키지 여행 유럽 관광지 순위첫 유럽 여행 추천 유럽여행 10일 코스 유럽 여행 추천 나라 유럽 여행 코스 짜기 유럽여행 필수코스 유럽 패키지 여행 비용 아프리카 아프리카 패키지 여행 아프리카 여행 코스 아프리카 여행 비용 공포스런 아프리카 여행 후기 아프리카 여행지 아프리카 사파리 여행 아프리카 배낭여행 진실'
    ,'미주·중남미' : '미국 여행 코스미국 자유여행 미국여행 패키지 미국 서부 여행 미국 여행 추천 도시 미국 여행 계획 미국 여행 비용 미국 뉴욕 여행 남미여행 루트 남미 추천 여행 남미 장기 여행 라틴아메리카 남미 지도 남아메리카 여행남아메리카 관광지'
    ,'오세아니아' : '오세아니아 관광지 오세아니아 여행 오세아니아 나라'
    ,'축구·풋살' : '풋살 축구 체력 풋살vs축구 축구와 풋살의 공통점 풋살 규칙 아이엠그라운드 풋살 풋살 매칭 풋살 기본기 풋살 용병'
    ,'야구' : '야구일정 야구순위 야구 생중계 야구결과 야구중계 오늘야구 네이버 야구 한국야구 해외야구'
    ,'농구' : '한국농구 농구 포지션 농구 규칙 NBA 농구 KBL 농구 프로농구 농구게임 농구 뜻'
    ,'골프' : '폭스바겐 골프 골프용어 골프 대회 골프레슨 골프경기중계 골프 역사'
    ,'테니스' : '테니스 입문 테니스 레슨 테니스 학원 테니스라켓 테니스 선수 테니스 대회 테니스 배우기 테니스 규칙'
    ,'헬스' : '헬스갤러리 헬스 영어로 헬스 초보 헬스 운동 헬스장 무서움 휘트니스 헬스 차이 헬스기구 헬스클럽'
    ,'요가·필라테스' : '요가 필라테스 요가 필라테스 장단점 요가 필라테스 가격 요가 필라테스 차이점 요가 필라테스 병행 요가필라테스 갤러리 요가 대 필라테스 플라잉요가 필라테스'
    ,'크로스핏' : '크로스핏 관절파괴 크로스핏 강도 크로스핏 몸 변화 크로스핏 현실 크로스핏 가격 크로스핏 크로스핏 초보 크로스핏 운동 종류'
    ,'스키·보드' : '스키보드 숏스키 스키보드 가격 스키 보드 난이도 스키 보드복 스키 장비 스케이트보드 스노우보드 바인딩 보드코리아'
    ,'스케이트' : '아이스스케이트 스케이트보드 롤러스케이트 피겨스케이트 스피드스케이트 스케이트 가격 아이스스케이트 신발 스케이트 종류'
    ,'수영' : '수영 근력운동 수영 운동 루틴 수영 몸 변화 수영의 장점 단점 수영 운동 효과 수영 운동량 수영 스포츠 수영선수 수영 방법'
    ,'런닝' : '매일 3km 달리기 효과 러닝 부작용 러닝 근력운동 러닝 다이어트 효과 매일 5km 달리기 효과 러닝 효과 매일 3km 달리기 다이어트 러닝 자세'
    ,'기타 스포츠' : '수구 배드민턴 바둑 사격 레슬링 탁구 펜싱 배구 스쿼시 태권도 사이클링 등산 카누 카약 미식축구 esport'
    ,'PC·노트북' : '티몬 pc버전 노트북 데스크탑 노트북 컴퓨터 부품 특가 PC부품 조립 컴퓨터 생활가전 조립PC'
    ,'PC 주변기기' : '공디스크/보관함 공유기 그래픽카드 네트워크장비 데이터/통신케이블 마우스 멀티탭 메인보드 모니터주변기기 소프트웨어 스위칭허브 영상사운드카드 중고 컴퓨터부품 컴퓨터 케이스 쿨러 키보드 타블렛 파워서플라이 프리젠터 조이스틱 CPU KVM스위치/액세서리 PC스피커 PC헤드셋/마이크 RAM USB액세서리 USB허브 음성케이블 전원케이블 젠더 IP카메라/CCTV'
    ,'스마트폰·태블릿' : '갤럭시케이스 기기일시불약정가입 기기할부약정가입 기타통신상품 베가케이스 보호필름 스마트앱세서리 아이폰케이스 충전케이블/젠더 해외폰전용케이스 휴대폰거치대 휴대폰메모리카드 휴대폰배터리 휴대폰촬영용품 휴대폰충전기 공기계'
    ,'음향기기' : '노래방기기 라디오 마이크 블루투스스피커 블루투스아이폰 블루투스헤드폰 사운드바 상업/특수음향 소스기기 스피커액세서리 앰프기기 오디어 이어폰 헤드폰액세서리 일반스피커 일반이어폰 일반헤드폰 카세트 CDP 케이블 포터블HiFi MP3 홈시어터 확성기 HiFi스피커'
    ,'웨어러블 기기' : '스마트워치 스마트밴드 VR기기 기타스마트기기 위치추적기 스마트플러그 웨어러블용품'
    ,'기타 장치' : "기타 장치"
    ,'주방가전' : '식기세척건조기 살균소독기 냉장고 냉동고 가스레인지 간식메이커 믹서기 블렌더 빙수기/제빙기 식품제조기/건강식제조기/발효기 에스프레소머신 원액기 음식물처리기 전기그릴 전기레인지 전기밥솥 전기오븐 전기쿠커 전기포트 전자레인지 전자저울 중탕기/약탕기 진공포장기 캡슐커피머신 커피메이커 토스터기 튀김기 핸드블랜더'
    ,'생활·미용가전' : '고데기/매직기 구강세정기 다리미 드라이기 무전기 미싱/재봉틀 사무가전 전기면도기 전기용품 전동칫솔 제모기 중고 소형가전 칫솔살균기 피부관리기기 헤어롤/에어브러쉬 헤어케이기기'
    ,'영상가전' : '리모컨 시스템TV 영상어댑터 프로젝터 프로젝터액세서리 해외직구TV 3D안경 DVD 블루레이 Divx TV안테나 TV액세서리 삼성TV 셋톱박스 LGTV 영상케이블'
    ,'세탁기·건조기' : '식기세척건조기 식품건조기 스타일러 의류건조기 세탁기용품 일반세탁기 드럼세탁기 탈수기 자동빨래삶통'
    ,'청소기' : '로봇청소기 무선청소기 물걸레청소기 보풀제거기 업소용청소기 진공청소기 청소기부품 스팀청소기 침구청소기 핸디청소기'
    ,'계절가전' : '습도조절기기 공기청정기 난방매트 난방기기 냉방기기 에어컨 에어컨용품 해충퇴치기'
    ,'기타가전' : "기타가전"
    ,'물리' : '고급 물리학 공통 물리 기초물리학 물리 실험 물리학 연구 가이드 물리학1 물리학2 빛의 성질 실험 상대성 이론 생물리학 소리의 성질 실험 쉬운 물리 심화 물리학 음향학 전기의 성질 실험 전자석의 실험 힘의 작용과 운동'
    ,'화학' : '고급 화학 금속의 산화 산과 염기 실험 산소와 이산화탄소의 성질 심화 화학 용해와 용액 실험 전해질 용액의 성질 화학 실험 화학1 화학2'
    ,'생물·생명' : '생명과학1 생명과학2 심화 생명과학 고급 생명과학 생명과학 실험 해부학 생화학 발생학 유전학 세포생물학 분자생물학 미생물학 해양생물학 동물학'
    ,'지구과학·천문우주' : '지구과학1 지구과학2 심화 지구과학 고급 지구과학 지구과학 실험 천체물리학 태양계'
    ,'전기·전자' : '논리설계 자기학 전자기학 전자회로 프로그래밍방법론 회로이론 전기전자 전기시스템선형대수 자료구조 아날로그전자회로 신호 및 시스템 심화 전기에너지변환 제어공학개론 반도체소자 확률변수 및 확률과정의 디지털 시스템 설계 통신의 운영체제 컴퓨터조직론 양자역학의 응용 전력 및 에너지시스템 알고리즘 전기전자 전기공학설계 컴파일러 임베디드시스템설계 나노소자 광전자공학 디지털집적회로 생체계측 유기전자소자 전자물리 전력전자공학 마이크로시스템기술개론 전력시장이론 전기기기 및 제어 로봇공학개론 최신제어기법 지능시스템개론 디지털신호처리  데이타통신망 전파공학 통신시스템 생체전기정보공학 네트워크 프로토콜 일본 신 전기전자 산업기술 전망 전기·정보공학개론'
    ,'기계공학' : '다공성물질 문제연구 인간중심 재활 및 보조기기 설계 기계공학 센서기반 위치 및 공간정보 추정 스마트 제조 및 응용 분자세포조직역학 기계나노공정개론 연소공학원론 정밀기계시스템설계 에너지 시스템 모델링 첨단제조공정 다상유동 레이저 이용 계측 연료전지개론  터보기계 전산유체역학 난류유동 기계공학수치해석 마이크로유체역학 고급가스터빈론 점성유동 비점성유동 저온공학 복사열전달  대류열및물질전달 고급내연기관론 고급기계공학해석 고급열역학 통계기반 공학해석 및 설계 지능재료 및 설계 전산나노역학 판과 쉘 전산해석 탄성학 유한요소해석 소성역학 연속체역학 견실최적설계방법론 제조고려설계 정밀기계계측 입자 및 에어로졸공학 바이오마이크로기전시스템 생체모사로봇공학 진동학 벡터공간최적화 스마트자율주행 시스템 제어 동역학 제어 로보틱스 특론 로봇역학 기초음향공학 로봇 액츄에이션 및 센싱 메커니즘 설계'
    ,'토목공학' : '창의공학설계 토목공학개론 유체역학 재료역학 토목지질공학 토질역학 철근콘크리트공학 토목시공학 토질역학 하천수리학 강구조공학 응용동역학 건설계측 매트릭스구조 상하수도공학 토류구조물설계 PS콘크리트 수공구조물 암반및지하공간설계 해안 및 항만공학 SOC종합설계'
    ,'화학공학' : '화학공학 물리화학 유기화학 공정모델링 공학수학 공학계산프로그래밍 나노입자공학 반응공학 화공열역학 화공유체역학 화공재료 공정제어 공학수치해석 열전달 분리공정설계 고분자공학 물질전달 생물화학 전기화학 화공안전공학 바이오재료 반도체공학 석유촉매화학공학 에너지공학'
    ,'역사' : '한국사학 동양사학 서양사학 동남아시아의 역사 독일의 역사 한국의 역사 고려 왕조의 사관 러시아의 역사 미국의 역사 영국의 역사 일본의 역사 중국의 역사 프랑스의 역사'
    ,'미술' : '동양화 서양화 조소 공예 디자인 도예 금속공예 시각디자인 산업지다인 판화 예술경영원론 미술경영 미술관 예술재원 미술품 지적 소유권'
    ,'음악' : '서양음악사 악전 화성학 대위법 악곡의 형식 연주이론 공연예술 조선후기음악 국악비평론 고음악연주 바로크음악 현문헌분석'
    ,'철학' : '중국철학 분석철학 심리철학 현대 인식론 프랑스철학 신존철학 현상학 서양근대철학 형이상학 성리학 불교철학 서양고중세철학 서양고대철학 윤리학 정치철학 법철학 신유학 중국선진제자백가 사회철학 분석철학 형이상학 근대독일철학 인식론 메타윤리학'
    ,'문학' : '한문학 음운사 현대시 현대소설 음운론 문법론 통사론 현대소설 고전시가 현대소설 및 비평 현대희곡 한시 음운론 텍스트언어학 문법사 의미론 고전소설 방언학 구비문학 통사의미론 한국어사 현대시'
    ,'양도소득세' : '양도소득세 신고납부기한 양도소득세 세액계산 흐름도 양도소득세 장기보유특별공제율 양도소득세 세율 양도소득세 가산세 양도소득세 신고납부안내 양도소득세 세액계산요령 양도신고서별제출증빙서류'
    ,'증여세' : '증여세 신고납부기한 증여세 세액계산 흐름도 증여세 가산세 증여세 신고시 유의사항 일감 몰아·떼어주기 증여의제 일감몰아주기 과세개요 일감떼어주기 과세개요 세액계산흐름도 증여재산의 평가 증여세 납부'
    ,'상속세' : '상속세 신고납부기한 상속세 세액계산흐름도 상속세 가산세 상속세 신고시 유의사항 상속재산의 평가 상속세 납부 상속재산의 확인 가업승계 지원제도'
    ,'종합소득세' : '종합소득세 모두채움 신고 안내 종합소득세 신고납부기한 종합소득세 세액계산 흐름도 종합소득세 세율 종합소득세 가산세 종합소득세 신고·납부방법 종합소득세 장부기장의무 종합소득세 기장의무 판단 종합소득세 간편장부  종합소득세 경비율적용 방법 종합소득세 성실신고확인제도 종합소득세 중간예납'
    ,'부가가치세' : '부가가치세 모두채움 신고 안내 부가가치세 신고납부기한 부가가치세 세액계산 부가가치세 세율 부가가치세 가산세 부가가치세 전자적 용역을 제공하는 국외사업자의 신고'
    ,'종합부동산세' : '종합부동산세 납부기한 종합부동산세 세액계산 흐름도 종합부동산세 세율 종합부동산세 가산세 종합부동산세 납부 종합 안내 종합부동산세 합산배제 및 과세특례 신고'
    ,'법인세' : '법인세 신고납부기한법인세 세율법인세 가산세법인세 신고시유의사항법인세 신고절차법인세 영리법인 신고 안내법인세 비영리법인 신고 안내법인세 공제감면법인세 중간예납법인세 동업기업 과세특례법인세 연결납세제도법인세 성실신고확인제법인세 연구인력개발비 세액공제 사전 심사제도'
    ,'연말정산' : '연말정산 신고안내 연말정산 중소기업취업자 소득세 감면안'
    ,'취득세·등록세' : '부동산 취득세 신고납부 차량취득세 신고납부 자동차세 연납 주민세 신고납부 지방소득세 신고납부 지방세 인터넷 서면신고 법인지방세업무처리'
    ,'세무조사·불복' : '과세전 적부심사 세금부과 이의신청 감사원 심사청구 조세심판원 심판청구  국세청 심사청구 조세심판원 기각결정 행정소송'
    ,'기타 세금상담' : "종교인소득 사업장현황신고 원천세 지방세"
    ,'회계자문' : '세무신고대행 법인세신고서·법인세무조정계산서 작성대행과 중간예납신고대행 임직원 소득세 원천징수이행신고와 종합소득세 신고납부 대행 부가세 신고납부대행과 전자세금계산서 작성관리대행 소득세 신고납부 원천징수이행신고서 각종소득세금 신고납부관리 상속세 증여세 양도소득세 신고서 작성과 납부대행 조세전략수립과 절세자문 법인조세전략수립과 세무경영의견서 작성 회계장부기장과 지도 및 관련 납세신고 기업의 장단기절세전략수립과 이익유현화전략설계 기업회계이익과 세무소득의 조정과 합의 조세불복 및 납세소송 과세적부심 세금이의신청 심사청구 심판청구 등의 조세불복 세금행정소송의 전단계와 전심절차 대행과 환급업무 이행 국세예규 유권해석 질의답변 사후보완 등 납세방어·예방업무 국제조세 이전가격조세 국내외 투자와 진출관련 세무자문 세무조사지원 세무조사 현장입회대응과 의견쟁점지원 서비스 사전세무조사 세무진단 예비세무조사 각종절세전략 수립 기업상속·증여·양도 가업승계 절세전략수립 창업자의 exitplan 은퇴전략 기업오너를 위한 CEO plan 퇴직금 반영과 설계전략 기업구조조정 재구축 관련 세무자문과 가업승계방법 부동산 취득 보유 운영 개발 양도 상속 증여 등의 세무·재무자문'
    ,'경제정책' : '거시경제 안정화 정책 교역 정책 경제성장을 창줄하기 위해 만들어진 정책 개발경제학과 관련한 정책 소득 재분배 관련 정책 규제 정책 반독점 정책 산업 정책기술 기반 경제 개발 정책'
    ,'경제동향' : '빅맥 지수 경제 지수 인플레이션 기본적 분석 GDP 거시경제학 미시경제학 ISLM 모형 NAIRU 실업률 이직률 주택 착공 건수 소비자 물가지수 소비자 레버리지 비율 산업 생산 지수 파산 국내 총생산 광대역 인터넷 보급률 소매 판매 지수 주식 시장의 주가 통화량'
    ,'경제용어' : 'EMBA 가격 대비 성능 가격 투명성 가격연동제 갈라파고스화 감각적 소비 개방경제 거시적 분석 게이레쓰 경기 경제동물  계절성산업 계정 고용감소 성장 고전학파의 이분법 골디락스 경제 공급의 법칙 관여도 구독 경제 국제경제무역 국제재무분석사 균형성장 금 선호 금유출 넌 제로섬 게임 닌자 론 대마불사  대한민국의 무역수지 데드 캣 바운스 델크레데레 도매물가지수 독점자본주의 동조 소비 등가교환 로트카볼테라 방정식 리보 맞춤화 미시적 기초 미코노미 반기 보세가공 무역 봉쇄경제와 개방경제의 비교 분수 효과 사설 교환 사회 간접 자본 상업어음 생계비지수 생산 공정 관리 생산수단의 사회화 생활 수준 생활수준연구 선호 선후관계와 인과관계 손금 손실 수단 목적 사슬 수신  수요와 공급 수요의 법칙 수익형 민자사업 수정자본주의 슘페터 장기 파동 가설 승수효과 승자독식시장 시가총액 시재금 신데렐라 전략 실증경제학 십분위분배율 어음 엥겔의 법칙 여신 외부성 원본 웰빙 유동성 위험 유보율 유상증자 유치산업 유통경제 이익중재 익금 인코텀스 임대형 민자사업 잉여가치 잉여금 자가경제 자연 가격 재고 조정 재무위험관리사 전자 어음 전자출결시스템 절약의 역설 정보 밀도 종합소득세 중간 이윤 지대추구 지수 진입 장벽 집적 이익 채권보증회사 초자본주의 총외채 추가경정예산 출자 출자총액제한제도 태환권 통화량 펀더멘털 폐쇄경제 풍선 효과 프리코노믹스 피구 효과 하방경직성 화폐수량설 환 평가 환업무 휴면계좌 흑자와 적자 희소성 히든 챔피언'
    ,'예금·적금' : "보통예금 당좌예금 저축예금 예금봉쇄 외화예금 요구불예금 저축성예금 예금통화 정기적금 자유적금 적금이자 예금이자"
    ,'주식·가상화폐' : '비트코인 P2P 가상화폐 암호화폐 소수주주권 비트코인투기 공개키 OTC 장외 거래 블록체인 채굴 거래소 소수주주권 보통주식 우선주식 주권 액면주식 무액면주식 기명주식 무기명주식 상환주식 전환주식 자기주식 주식매수선택권 주가 가치투자 모멘텀투자 선물 옵션 HTS 차익거래 비차익거래 플래시 트래이딩 유령주식 하이프리퀀시 트레이딩'
    ,'대출' : '학자금 대출 P2P 대출적격전환 대출 대출금리 신용대출 건별대출 유동성한도대출 일시상환 분할상환 고정금리 변동금리 서류 심사 예금담보대출 신용대출 전세자금대출 부동산담보대출 주택도시기금대출 자동차대출 기업대출'
    ,'자산관리' : '저축상품 대출상품 금융투자상품 자산배분 수익률 위험관리 재무설계 상호저축은행 적금 은행 특판 상품 CMA MMF 펀드 인덱스펀드 연금 변액보험 위험헷지 비과세'
    ,'부동산' : '부동산투자회사 주택저당증권 채권담보부증권 자산유동화증권 담보인정비율 총부채상환비율 총부채원리금상환비율 전용면적 건폐율 용적률 단독주택 다가구주택 연립주택 다세대주택 도시형 생활주택 오피스텔 빌라 재건축 재개발 도시개발사업 뉴타운 보금자리주택 장기전세주택 뉴스테이 공공임대아파트 국민인대주택'
    ,'무역' : '도착통지서 전위험담보 미 관세청의 적하목록시스템 최저운임제 항공화물운송장 선하증권 유류할증료 정정통지서 상업송장 원산지증명서 용선계약서 통화할증료 입방미터 긴급유류할증료 착지불수수료(항공) 운임포함조건(C&F) 컨테이너화물 집화소 운임보험료포함조건 컨테이너 장치장 화물인도지시서 도착지화물인도비용 관세지급 반입인도조건 관세미지급 반입인도조건 수출면허(승인) 긴급유류할증료 전자문서교환 입항예정일(예상도착일) 출항예정일(예상출발일) 공장인도조건 운임목록 유류할증료 컨테이너 유류할증료(항공) 국제복합운송업협회 본선인도조건 자유무역협정 공동해손 관세및무역에관한일반협정(제네바협정) 기본(일반)운임인상 국제통일상품명 및 코드시스템 수입신고(서) 수입승인(서) 보험증권 국제항공운송협회 내륙컨테이너기지 IMO 위험물분류코드 국제해사기구 국제통화기금 무역조건 해석에 관한 국제규칙 내륙지역 복합운송 서비스 국제표준화기구 한국복합운송주선업협회 신용장 화물선취보증서 손상화물보상장(면책증서) 소량컨테이너화물 의향서 적하목록 본선수취증 적하목록취합시스템 북미대륙횡단(철도) 서비스 양해각서 복합운송인 물질안전보건자료 무선박운송인 포장명세서 단독해손 성수기할증료 운임적용톤 냉동컨테이너 자동차 전용적재 선박 우대운송계약 선적일(무역최종선적일) 선적지시서 선적요청서 특정품목운임율 화주소유 컨테이너 정기용선 화물대도증 환적 전신환 운항소요시간 터미널핸들링차지 전손 화물배상책임보험조합 아시아/북미수입운임협정 항차 선박 부두사용료 분손담보 중량톤 세계무역기구'
    ,'근로계약' : '근로청소년 청소년 아르바이트 청소년 근로가능 연령 취직인허증 청소년 근로금지업소 갱내근로 정규직 전환 계약직 청소년고용금지업소 청소년 출입ㆍ고용금지업소 연소자 증명서 현장실습표준협약서 근로계약 근로계약의 대리 금지 근로계약의 취소 근로계약의 일부 무효 연소근로자 표준근로계약서 최저임금 최저임금의 적용대상 청소년의 근무 가능시간 야간ㆍ휴일근로의 제한 근로조건의 서면명시 인턴'
    ,'휴일·휴가' : '근로시간 휴가권 보장 근로시간 단축 배우자 출산휴가 배우자 출산휴가 신청 배우자 출산휴가 급여 가족돌봄 지원 가족돌봄휴직 가족돌봄휴가 가족돌봄 등을 위한 근로시간 단축 양육 지원 육아휴직 및 육아기 근로시간 단축 육아휴직 육아휴직 신청 육아휴직 급여 육아기 근로시간 단축 육아기 근로시간 단축 신청 육아기 근로시간 단축 급여 고용안정장려금'
    ,'구조조정' : '분사 기업 채무 채권자 일시 파산 해고 자율협약 기업재무구조개선 기업회생절차 구조조정 희망퇴직'
    ,'해고·징계' : '해고 근로관계의 종료와 해고 해고의 의의와 판단기준 경영상 이유에 의한 해고 근로기준법에 따른 해고의 제한 및 적용범위 해고 사유의 제한 해고 사유 및 정당한 이유의 해고 노동관계법령상 금지되는 해고 사유 정당한 이유 없는 해고의 금지 해고 사유의 정당성에 대한 구체적 판단 경영상 이유에 의한 해고의 정당성에 대한 구체적 판단 해고 시기와 절차의 제한 해고예고 해고 사유 등의 서면 명시 단체협약ㆍ취업규칙에 따른 절차 준수 부당해고와 구제절차 부당해고의 의의와 구제신청 지방노동위원회에 대한 부당해고 구제신청 지방노동위원회 구제명령 등에 대한 재심 법원에 의한 부당해고 구제(행정소송) 중앙노동위원회의 재심판정에 대한 행정소송 해고무효확인의 소(민사소송) 해고근로자 보호 취업방해금지 등 임금, 퇴직금 등 지급 금품청산 퇴직금의 지급 미지급 임금ㆍ퇴직금(체불임금) 구제 절차 임금채권의 지급보장(대지급금) 해고근로자 지원 실업급여 구직급여 취업촉진 수당 실직자 지원제도 실업자 등에 대한 직업능력개발훈련 직업훈련 생계비 대부'
    ,'산업재해' : '개관 근골격계질병의 유족보상연금 국외 재해의 연금 비용 화학적 요양비 감염성 장해등급의 유족보상일시금 산업재해보상 요인에 재활지원 사고로 장해에 요양급여 청구 휴업급여 중소기업 장해특별급여 급성 산재보험 상병보상 림프조혈기계 직업성 직업훈련비용 재해 재요양 요양 산업재해보상보험 인한 중독 신경정신계 판정 보장법상의 장해등급 상병보상연금 암의 그 해외파견자에 부분휴업급여 장해급여 특수형태 사업에 피부, 및 조정 의한 변경 현장실습생에 본인 특례 행사 지급 물리적 중 중의 호흡기계, 진폐에 의료기관 가중 인정기준 기간 부담금 국민기초생활 직업재활급여 근로종사자에 유족특별급여 추정에 안에서의 사업장 이송비 질병의 연장 밖에서의 심장질환의 확장/축소닫기산업재해보상보험 간, 장해급여의 직업훈련수당 근로자의 또는 사고 업무상 장례비 눈 산업재해보상보험법 요양기간의 지원(후유증상관리) 보험급여 의료재활 건강손상자녀에 대부 따른 귀 대한 사업주등에 뇌혈관질병 밖의 저소득 사망 재요양에 등 수급자에 유족급여 재판정에 질병 간병료 중증요양상태등급의 고령자의 일부 직장복귀지원금'
    ,'직장내괴롭힘' : '직장내 폭행 및 부당지시 직장내 차별 직장내 명예훼손 외 직장 모욕 직장 내 따돌림 업무 직장내 폭언 내 강요'
    ,'임금체불' : '체불임금 시효 도급사업 간이대지급금 비상시 대지급금 임금채권우선변제 최저임금 지급 임금명세서 외국인근로 임금체불신고 금품청산 임금대장 통상임금 휴업수당 임금지급 도산대지급금 임금 체불사업주 임금체불'
    ,'임금·급여' : '건설일용근로자 건설업 일용직 건설일드림넷 취업지원센터 유료직업소개소 부당한 소개 금지 거짓구인광고 근로자파견 최저임금 도급인 연대책임 직상 수급인 법정근로시간 근로계약 근로조건 근로계약 일부 무효 표준근로계약서'
    ,'기타 노무상담' : '노무상담'
    ,'하이킹' : '하이킹 운동화 하이킹옷 하이킹 모임 하이킹 카페 하이킹'
    ,'등산·클라이밍' : '등산 용품 등산복 등산 장비 등산카페 등산모임 등산스틱 온싸이트 플래싱 레드포인트 요요잉 행도깅 톱로핑 슬랩 페이스 침니 촉스토운 크랙 핑거크랙 핸드크랙 벙어리크랙 좌향크랙 우향크랙 직상크랙 리스 디에드르 칸테 릿지 걸리 꿀루아르 렛지 테라스 오버행 루프 버트레스 스탠스 밴드 나이프 엣지 풋 홀드 토우스텝 프릭션스텝 사이드스텝 프릭션토우스텝 풋잼 풋 앤드 니이 잼 핸드 홀드 그립홀드 핑거홀드 인서어트홀드 언더홀드 푸시홀드 네일홀드 포켓홀드 내면 핸드잼 양손잼 주먹잼 양손 어퍼지션 핑거어퍼지션 무브 크럭스 빌레이 보울더링 힐 훅 레이백 펜듀럼 트레버스 압자일렌 클라이밍 다운 안자일렌 격시등반'
    ,'낚시' : '출조 동출 조사 조과 꽝조사 대상어 씨알 빵 만쿨 조황 런커 밀물 썰물 갯바위 낚시 루어낚시 릴 찌낚시 원투 낚시 지깅낚시 트롤링 낚시 좌대낚시 캐스팅 히트 액션 저킹 폴링 바이트 훅셋 랜딩 리트리브 릴리즈 스키핑 리프트&폴 호핑 쉐이킹 트위칭 드래그 숏바이트 두들링 어프로치 프레셔 로드 릴 라인 노브 슬랙라인 루어 미노우 베이트 베이트피쉬 하드베이트 펜슬베이트 포퍼 탑워터 차거 플로팅 블레이드 콤비네이션루어 스풀 드랙 드랙노브 릴풋 백래시 퍼머 서밍 플라이어 태클 태클박스 쿨러 스피닝릴 타이라바 헤드 훅 스커트 다운샷 다운샷리그 래틀 싱커 노싱커 비드 옵셋훅 슬랙라인 스피너베이트 스플리샷리그 텍사스리그 와키리그 위드가드 물색 청물 뻘물 갯바위 곶부리 바텀 백워터 브레이크라인 생추어리 스쿨링 피딩타임 하비스트 스폰 쇼어라인 오픈워터 쉘로우 스팟 엣지 캄사이드 물고기 물가 대어'
    ,'캠핑' : '가렌더 그늘막 텐트 그라운드 시트 방수포 노지 때캠 데크 릴렉스 체어 모캠 미니멀 캠핑 캠핑 매너타임 캠핑 바닥공사 비박 백패킹 불멍 스테킹 써큘 스트링 라이트 스토퍼 옆지기 오지 알빙 이소가스 차박 피칭 파워뱅크'
    ,'책·독서' : '통독 속독 정독 음독 묵독 발췌독 표면 읽기 행간 읽기 수용적읽기 비판적읽기 속담 명언 시 수필 소설 만화책 베스트샐러 아동도서 독서토론'
    ,'사진·영상' : "자연사진 풍경사진 천체사진 폭풍사진 반려동물사진 꽃사진 건축물사진 매물사진 드론사진 항공사진 인물사진 얼굴사진 패션사진 스포츠사진 다큐멘터리사진 거리사진 웨딩사진 음식사진 제품사진 정물사진 흑백사진 예술사진 이중 노출사진 초현실사진 추상사진 자연영상 풍경영상 천체영상 폭풍영상 반려동물영상 꽃영상 건축물영상 매물영상 드론영상 항공영상 인물영상 얼굴영상 패션영상 스포츠영상 다큐멘터리영상 거리영상 웨딩영상 음식영상 제품영상 정물영상 흑백영상 예술영상 이중 노출영상 초현실영상 추상영상 자연영상 풍경영상 천체영상 폭풍영상 반려동물영상 꽃영상 건축물영상 매물영상 드론영상 항공영상 인물영상 얼굴영상 패션영상 스포츠영상 다큐멘터리영상 거리영상 웨딩영상 음식영상 제품영상 정물영상 흑백영상 예술영상 이중 노출영상 초현실영상 추상영상 "
    ,'음악' : '클래식 재즈 CCM 팝 발라드 블루스 힙합 컨트리 뮤직 포크 음악 레게 디스코 록 음악 전자 음악 트로트 일렉트로닉 뮤직 로큰롤 노래'
    ,'가드닝' : '식물키우기 반려식물 정원 빛 흙 통품 물주기 식물심기 식물가꾸기 정원가꾸기 화분 화초'
    ,'대인관계' : '타인을 지나치게 통제하거나 조종하려는 경향 적대적 지배성 타인에게 쉽게 화를 내고 타인에 대한 불신과 의심을 갖는 경향 타인에 대한 정서적 경험과 표현 친밀성이 어려운 냉담성 여러 사람 앞에서 불안해하고 당황하며 매사에 소심한 경향 타인과의 관계에서 자신감 결여 욕구 및 의사표현의 어려움 타인과의 관계에서 독립성 유지에 어려움 쉽게 설득당하며 지나치게 순종적인 정도 타인의 욕구에만 지나치게 민감하고 자신의 처지는 아랑곳하지 않는 경향 타인과의 관계에서 강한 결속력을 요구하고 자신에게 항상 관심을 기울여주길 바라는 성향 대인관계에서 경계의 부족 타인의 비판에 대한 민감성 타인의 입장을 수용하고 공감하는 태도 현재 타인에 대한 분노나 복수심 적대감 타인에 대한 지나친 관심과 인정욕구 타인과의 어울림에 대한 불편 자기표현능력의 부족'
    ,'자아·자기이해' : "자신감 자아존중감 자아개념 발달 충동감소이론 동기 신행동주의 서로다른 충동 평가 경험 성공과 실패경험에 의한 자아개념"
    ,'연애·이성' : '남녀 입장에서의 연애 및 결혼 연애 시작의 어려움 사귀기 전 단계 또는 썸 남녀 간의 호감 신호 배경 외모 학벌 직업 등의 문제 미성년자 시기의 연애 성년 이후의 연애 금전 문제 어려운 인간관계로서의 연애 동성연애'
    ,'정신건강' : '우울증 불면증 조울증 조현병 화병 무기력증 정신과의 심리학 주의집중 곤란 강박적 행동'
    ,'기타 심리상담' : '사주 운세 타로카드 삶에 대한 회의 가치관의 혼란 죽음에 대한 충동'
    ,'중학교 생활' : "중학교 학생 공부 학원 시험 학교 시간 영어 수업 교육 입학 선생님 단원 입시 친구 연애"
    ,'고등학교 생활' : '고등학교 학생 수능 생활 학교 공부 친구 대학 과목 학습 학원 전형 진로 수시 정시 고사 시간 수학 선생 생각 공부 생각 운동'
    ,'대학교 생활' : "회사 대학교 삶 인생 대학생 친구 학교 활동"
    ,'회사 생활' : "퇴사 취업 직원 월급 출근 직장인 마음 사회 이직 동기 사회"
    ,'군대 생활' : '군인 군대 선임 후임 입대 병영심리상담 자살위기 성인지력 스트레스 분노 진로상담'
    ,'연애·결혼' : "애정운 자녀 가족건강 진로 결혼 예비부부 신혼부부 바람 불륜 육아 고부갈등"
    ,'기타 고민상담' : "기타 고민상담"
    ,'김연경 선수' : '김연경배구 김연경모델 김연경경기 김연경강타 김연경남편 김연경연봉 김연경 결혼 사진 김연경 세례명 김연경 고향 김연경 근황 김연경 가족'
    ,'표창원 소장' : '표창원부인 직업 표창원 재산 표창원 아들 표창원 교수 표창원 근황 표창원 고향 표창원 하차 표창원 경정'
    ,'황대헌 선수' : '황대헌메달리스트 황대헌 올림픽 황대헌 동계올림픽대회 황대헌 쇼트트랙 황대헌 국대'
    ,'이연복 셰프' : '이연복중국집 이연복 재산 이연복 대표 이연복 방송 이연복 요리 이연복 협업 이연복 직원 이연복 영상편지'
    ,'홍성우 원장' : '홍성우비뇨기과 홍성우 전문의 닥터조물주 홍성의 대표원장'
    ,'김초엽 작가' : '김초엽책 김초엽 작가 김초엽 소설 김초엽 이야기 김초엽 작품 김초엽 독서 김초엽 지구 김초엽 미래 김초엽 단편 김초엽 인간 김초엽 영화'
    ,'백세희 작가' : '백세희 작가 백세희 저서'
    ,'의료 보험' : '질병이나 부상으로 인해 발생한 고액의 진료비로 가계에 과도한 부담이 되는 것을 방지하기 위하여 국민들이 평소에 보험료를 내고 보험자인 국민건강보험공단이 이를 관리·운영하다가 필요 시 보험급여를 제공함으로써 국민 상호간 위험을 분담하고 필요한 의료서비스를 받을 수 있도록 하는 사회보장제도'
    ,'상해 보험' : '일상생활도중 발생할 수 있는 급격하고도 우연한 외래의 사고로 인해 신체에 상해를 입었을 경우 그 손해를 보상해 주는 보험으로 사망 후유장해보험금 의료비 등을 지급하는 보험'
    ,'저축성 보험' : '저축시 지급되는 보험금의 합계액이 이미 납입한 보험료를 초과하는 보험으로 위험보장과 저축기능을 겸비한 보험'
    ,'재산 보험' : '화재 등으로 인해 보험의 목적인 재물에 생긴 손해를 보장하는 보험'
    ,'교통사고 과실' : '운전 중 발생할 수 있는 각종 신체상해 위험과 법률관련 비용 및 기타 비용손해를 종합 보장하는 보험'
    ,'식습관·식이요법' : '식이요법은 건강 회복 및 증진을 목적으로 식단을 조절하는 것으로 다이어트에 대한 수행법 당뇨 체중조절을 위한 식이요법 당뇨병 치료용 식이요법 고혈압 치료용 식이요법 PKU 치료용 식이요법 크론병의 식이요법 알레르기 치료 식이요법'
    ,'다이어트 식단' : '식욕억제제 삭센다 위고비 마운자로 오젬픽 고혈압 고지혈증 당뇨병 고도비만 내장지방 긁지 않은 복권 린 시스템7 외모관리 피트니스 클럽 유산소 운동 무산소 운동 컨디셔닝 운동 다이어트와 영양소 마른비만 몸 긍정 미모 봉인구 내과 가정의학과 대사증후군 식이요법 비만 살이 찌지 않는 체질 섭식장애 올리스탯 요요 현상 지방흡입 수술 체성분 검사 체중 감량 애프터번 NEAT 대한비만학회 프로틴플레이션 미네소타 기아 실험'
    ,'영유아·아동 식단' : '이유식 모유 분유 건강한 메뉴 특수식단 알레르기 식품 불내성'
    ,'환자 식단' : "유동식 전유동식 미음식 식욕부진"
    ,'기타 영양상담' : '영양상담'
    ,'드라마' : "한국드라마 미국드라마 중국드라마 영국드라마 대만드라마 일본드라마 군대드라마 막장드라마 모노드라마 미니드라마 역사드라마 단편드라마 농촌드라마 정치드라마 판타지드라마 웹드라마 특촬물 수사물 에로"
    ,'예능' : '코미디 프로그램 토크쇼 리얼 버라이어티 관찰 버라이어티 오디션 서바이벌 프로그램 시청자 참여 듀엣 음악 프로그램 쿡방 먹방 프로그램 '
    ,'영화·애니' : '액션 영화 범죄 영화 SF 영화 코미디 영화 슬랩스틱 코미디 영화 로맨스 코미디 스릴러 영화 공포 영화 전쟁 영화 스포츠 영화 판타지 영화 음악 영화 뮤지컬 영화 멜로 영화 형식 및 양식 무성 영화 흑백 영화 유성 영화 특수 촬영 영화 3-D 영화 단편 영화 장편 영화 기타 컬트 영화 교육 영화 선전 영화 실험 영화 B급 영화 풍자 영화 패러디 영화 저예산 영화 엑스플로이테이션 영화 블랙스플로이테이션 독립 영화 애니메이션 영화 애니 GL 애니 BL 애니 하렘 애니 역하렘 애니 일상 애니 먼치킨 애니 TS 애니 리치 애니 집사 애니 이세계 애니 좀비 애니 오타쿠 애니 성장 애니'
    ,'웹툰·웹소설' : '스포츠물 국내 GL 소설 액션/무협 만화 학원 BL 역사/시대물 판타지/SF 순정 현대물 TL 드라마 신무협 공포/추리 서양풍 게임 만화잡지 퓨전 라노벨 연재 스포츠 대체 웹소설 하이틴 역사물 스포츠/학원 할리퀸 전통 무협 라이트노벨 가상 로맨스 판타지 세계 현대 해외 정통 로판 웹툰 액션 판타지물 e북 동양풍 코믹'
    ,'뮤지컬·연극' : '가면극 신파극 변검 델라르테 인형극 비극 희극 탈춤 라쿠고 일본 마당놀이 뮤지컬 마술 코메디아 동물공연 쇼 중국 무언극 조루리 코미디 한국 포대희 쿠미오도리 캐릭터 남사당놀이 서양 레뷰 서커스 부조리극 프릭쇼 오페라 판소리 노가쿠 창극 가부키 경극 모노드라마 프로레슬링'
    ,'인테리어' : '프로방스 모던인 한국아시아 클래식앤틱 북유럽 MIX 빈티지 유니키 테리어 내추럴 인테리어' 
    ,'청소' : '준공청소 입주청소 이사청소 주거청소 부분청소 에어컨청소 세탁기청소 쓰레기집청소 화재청소 고독사청소'
    ,'세탁수선' : '일반의류 수선 세탁 와이셔츠 수선 세탁 이불 수선 세탁 운동화 수선 세탁 아웃도어 수선 세탁 교복 수선 세탁 모자 인형 가방 수선 세탁 사이즈 늘리기 줄이기 줄임 늘림 가죽 교체 고무줄달기 지퍼 달기 장식 보강'
    ,'생활꿀팁' : '생활꿀팁 생활속 꿀팁'
    ,'자동차' : '자동차 에어콘 고무호스 라디에이터 냉각계통 점검 자동차 전기계통 점검 자동차 스노우타이어 체인보관 자동차 세차 자동차 와이퍼 자동차 내부 습기 제거 자동차 배터리 점검 자동차 냉각수 자동차 연료통 호스 연료필터 연료계통 점검 자동차 엔진오일 점검 자동차 워셔 점검 자동차 브레이크 액 점검 자동차 변속기기 오일 점검 자동차 파워 스티어링 오일 점검'
    ,'양육·훈육' : '훈육 뜻 훈육 훈육 매 훈육 만화 아이 훈육 교사 훈육 훈육 폭력 양육 뜻 육아 양육 차이 양육하다 뜻 양육비용 동물 양육 보육'
    ,'놀이' : '유아놀이활동 유아기 놀이 종류 유아 놀이 활동 프로그램 유아 놀이 발달 단계 유아 놀이의 중요성 놀이의 중요성 6가지 유아에게 놀이란 놀이중심 교육과정'
    ,'유아교육' : '유아교육 트렌드 유아교육 프로그램 유아교육 중요성 유아교육 사이트 유아교육 이슈 유아교육이란 유아 교육 포털 유아교육키드키즈'
    ,'기타 육아상담' : '어린이집 상담 육아서비스 육아 분담 육아 바우처 육아 지식 육아정보 육아종합'
    ,'약 복용' : '올바른 약 복용 약 복용 시 주의사항 약 혼합복용 약복용 간격 약 복용 시간 간격 약 복용 시간 약 복용법 잘못된 약 복용'
    ,'영양제' : '종합영양제 순위 영양제 부작용 영양제 추천 영양제 효과 없다 영양제 사이트 영양제 종류 영양제 뜻 영양제 수액'
    ,'기타 약료상담' : '방문약료 지역사회약료 사회약료서비스 대한 약사회 전화 번호 다제약물 관리사업 Kaw kpanet or kr 대전시 약사회 홈페이지 Member kpanet or kr '
    ,'한약' : '한약처방 검색 한약처방명 한약 비싼 이유 한약 효과 있나요 한약 한달치 가격 한약 종류 한약 환 한약 효과 검증 '
    ,'전자기기 수리' : '계산기수리 기타수리 전자수리 제품수리 라디오수리 레이더수리 감지기수리 배터리수리 스마트수리 홈수리 기기수리 실외수리 온도계수리 알람수리 시계수리 컴퓨터수리 칩수리 정보수리 태양등수리 테스트수리 장비수리 텔레비전수리 헤드폰수리 홈수리 시어터수리 및수리 오디오수리 장비수리 E-Cigarette수리 Vaporizer/전자수리 담배수리 무화기수리 금융거래장비수리수리 GPS수리 미디어수리 플레이어수리 수리수리 PDA수리 휴대용수리 블루투스수리 스피커수리 수리수리 전문수리 오디오수리 장비수리 수리수리 로봇수리 레스토랑수리 서버수리 수리수리 추적수리 장치수리 수리수리 Virtual수리 Reality/가상수리 현실수리 헤드셋수리 음성수리 녹음기수리 수리수리 '
    ,'자동차 수리' : "교환 및 탈부착 모든 종류의 판금도장 작업 등 점검‧정비 및 튜닝작업"
    ,'가전제품 수리' : "오븐/전자레인지 수리  더 플레이트 인덕션(휴대용) 수리 선풍기 수리 냉장고 수리"
    ,'반려동물 건강' : "반려견 예방접종 강아지 미용 접종 사료"
    ,'반려동물 훈련' : '반려동물수리 훈련수리 상담사수리 반려동물수리 훈련사수리 강아지수리 훈련수리 종류수리 강아지수리 훈련수리 순서수리 강아지수리 개인기수리 종류수리 강아지수리 훈련수리 시기수리 반려동물수리 산책로수리 강아지수리 필수수리 훈련수리 '
    ,'반려동물 미용' : '애견미용사수리 현실수리 반려동물수리 미용사수리 자격증수리 반려동물수리 미용사수리 하는수리 일수리 애견미용사수리 월급수리 반려동물미용사수리 연봉수리 애견미용사수리 구인수리 애견미용사수리 나무위키수리 애견미용사수리 연봉수리 '
    }

### - 데이터 전처리

* 데이터 수집 프로세스 
1. 각종 사이트에서 수집
2. 여러기법으로 전처리

In [ ]:
import re


# 콤마제거
def replace_comma(word):
    words = []
    for i in word.split():
        words.append(i + "수리 ")
    words = "".join(words).replace(",", "")
    return words


# 콤마와 \n, \t 제거
def replace_comma_nt(word):
    word = word.replace(",", "").replace("\n", '').strip().replace("\t", "")
    return word

# 가운데 점제거
def replace_middle_dot(word):
    word.strip().replace("·", "")
    words = []
    for i in word.strip().replace("·", "").split():
        words.append(i + " ")
    word = "".join(words).strip()
    return word

# 특수문자 한자 제거
def replace_special_word(word):
    word = re.sub(r'[一-鿃]+', '', word)
    return word
    
# # 특정 키워드 추가후 출력(일회성)        
# for i in mrg_df[mrg_df['KoparentId'] == '기업']['title']:
#     print(f",'{i}' : '{i} 계열사 {i} 재무재표 {i} 매출액 {i} 평점 {i} 채용 {i} 연봉 {i} 복지 {i} 자소서 이력서'")
    

word = """· 구강악안면외과   · 턱 교정   · 소아청소년교정   · 양악수술   · 주걱턱   · 돌출입   · 긴얼굴   · 안면비대칭   · 무턱   · 보철과   · 보존   · 교정과   · 치과 교정학   · 성인교정   · 심미교정   · 수술교정  """
replace_middle_dot(word)


### - 초기 데이터와 수집데이터 맵핑

In [ ]:
# 기존 데이터프레임과 새로수집한 딕셔너리 title을 키로 잡고 merge
import pandas as pd
def mrg_dict_to_dataFrame(mrg_df, detail_title):
    global new_df, cnt_df
    new_df = pd.DataFrame(list(detail_title.items()), columns=['title', 'content'])
    cnt_df = pd.merge(mrg_df, new_df, on='title', how='inner')
    return cnt_df

In [ ]:
# 기존 데이터프레임과 새로수집한 딕셔너리 title을 키로 잡고 merge
# 최종 정제 데이터 *cnt_df
cnt_df = mrg_dict_to_dataFrame(mrg_df, detail_title)
len(cnt_df)

### - 정제 데이터 save
* pickle로 저장

In [ ]:
# cnt_data pkl로 저장 
import pickle
with open('cnt_data.pkl', 'wb') as f:
     pickle.dump(cnt_df, f)

### - 정제 데이터 load

In [ ]:
import pickle
with open('cnt_data.pkl', 'rb') as f:
    cnt_df = pickle.load(f)

### - 거리별로 ann 파일로 저장 
* angular, euclidean, manhattan, hamming, dot

In [ ]:
from sentence_transformers import SentenceTransformer, util
import pickle
import numpy as np
import pickle
from annoy import AnnoyIndex
import random
import time


# 임베딩 값 불러오기
embedder = SentenceTransformer('jhgan/ko-sroberta-multitask')
# embedder = SentenceTransformer('smartmind/ko-sbert-augSTS-maxlength512')
# embedder = SentenceTransformer('intfloat/multilingual-e5-base')
# embedder = SentenceTransformer('ddobokki/unsup-simcse-klue-roberta-small')
# embedder = SentenceTransformer('sentence-transformers/facebook-dpr-ctx_encoder-single-nq-base')
embedding_start = time.time()
sentences = cnt_df['content'].tolist()
corpus_embeddings = embedder.encode(sentences, convert_to_tensor=True)

vector_size = 768  # Length of item vector that will be indexed

model_name = ['jhgan','smartmind','intfloat','ddobokki','sentence_transformers']
metric_list = ['angular', 'euclidean', 'manhattan', 'hamming', 'dot']

for model in model_name:
    for metric in metric_list:
        
        annoy_index = AnnoyIndex(vector_size, metric=metric)
        for idx in range(cnt_df.shape[0]):
            
            vector = corpus_embeddings[idx]
            annoy_index.add_item(idx, vector)

        annoy_index.build(10) # 10 trees
        path = f'/Users/seunghyun/anaconda3/envs/my_env/23.09.25/ann/{model}'
        annoy_index.save(f'{path}/topic_classify_{metric}.ann')
        embedding_end = time.time()
        embedding_time = embedding_end - embedding_start
        print(f"저장경로 {path}")
        print(f"{model} | {metric} : 임베딩 및 ann 저장 {embedding_time} 초")     

### - 1차 구현 출력 상태 확인

In [ ]:
import time
from annoy import AnnoyIndex
import pickle
from sentence_transformers import SentenceTransformer, util



queries = '약1년6개월쯤 커피숍을 권리금을 받고 넘겼는데 시설물(에어컨 ㆍ커피머신등 시설비 )를 비싸게 받았다고 약간의 금액을 돌려달라고하는데 어떻게 하죠'
with open("cnt_data.pkl", 'rb') as f:
    cnt_df = pickle.load(f)
vector_size = 768
annoy_index = AnnoyIndex(vector_size, metric)
start_time = time.time()

# queries = input()
queries = [queries]

annoy_index.load('/Users/seunghyun/anaconda3/envs/my_env/23.09.25/ann/ddobokki/topic_classify_angular.ann')
# 3개만 출력
top_k = 230
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    top_results = annoy_index.get_nns_by_vector(query_embedding, top_k, include_distances=True)

    end_time = time.time()
    
    print("\n======================\n")
    print("검색어: ", query)
    print(f"검색결과 {top_k}개 출력")
    print("\n======================\n")
    
    for idx, distance in zip(top_results[0], top_results[1]):
        print(f"중분류 : {cnt_df['KoparentId'].iloc[idx]} | 소분류 : {cnt_df['title'].iloc[idx]} | {metric} distance : {round(distance,4)}")    
        
    print(" ")
    search_time = end_time - start_time
    print(f"검색 시간: {search_time:.4f} 초")
    
    # distance_df = pd.DataFrame({'KoparentId': cnt_df['KoparentId'].iloc[idx], "title": cnt_df['title'].iloc[idx], 'metric': round(distance,4)})
# annoy, 지표, 모듈화

---

In [ ]:
import time
from annoy import AnnoyIndex
import pickle
from sentence_transformers import SentenceTransformer, util
from sklearn.preprocessing import minmax_scale

queries = """ 오피스텔은 아파트랑 구조가 똑같은 건가요? 제가 평소에 오피스텔이 한 번씩 너무 헷갈리는데요 오피스텔은 혹시 아파트랑 구조가 완전히 똑같은 건가요? 그런데 왜 오피스텔이라고 따로 만드는 거죠?"""

# # (angular, euclidean, manhattan, hamming, dot)
# metric = 'angular'    
embedder = SentenceTransformer('jhgan/ko-sroberta-multitask')
# embedder = SentenceTransformer('smartmind/ko-sbert-augSTS-maxlength512')
# embedder = SentenceTransformer('intfloat/multilingual-e5-base')
# embedder = SentenceTransformer('ddobokki/unsup-simcse-klue-roberta-small')
# embedder = SentenceTransformer('sentence-transformers/facebook-dpr-ctx_encoder-single-nq-base')

def start(queries,metric,embedder):
    with open("cnt_data.pkl", 'rb') as f:
        cnt_df = pickle.load(f)
    vector_size = 768
    annoy_index = AnnoyIndex(vector_size, metric)
    start_time = time.time()

    # queries = input()
    queries = [queries]
    path = '/Users/seunghyun/anaconda3/envs/my_env/23.09.25/ann/smartmind/topic_classify_'+ f'{metric}' +'.ann'
    annoy_index.load(path)
    # 3개만 출력
    KoparentId_list = []
    title_list = []
    distance_list = []
    top_k = 230
    for query in queries:
        query_embedding = embedder.encode(query, convert_to_tensor=True)
            
        top_results = annoy_index.get_nns_by_vector(query_embedding, top_k, include_distances=True)

        end_time = time.time()
        
        print("\n======================\n")
        print("검색어: ", query)
        print(f"검색결과 {top_k}개 출력")
        print("\n======================\n")
        
        for idx, distance in zip(top_results[0], top_results[1]):
            print(f"중분류 : {cnt_df['KoparentId'].iloc[idx]} | 소분류 : {cnt_df['title'].iloc[idx]} | {metric} distance : {round(distance,4)}")    
            KoparentId_list.append(cnt_df['KoparentId'].iloc[idx])
            title_list.append(cnt_df['title'].iloc[idx])
            distance_list.append(round(distance,4))
            # print(round(distance,4))
            
        print(" ")
        search_time = end_time - start_time
        print(f"검색 시간: {search_time:.4f} 초")
        global result_df
        result_df = pd.DataFrame({'KoparentId': KoparentId_list, "title": title_list, f'{metric}_distance': distance_list})
        # 거리별 데이터 프레임을 만들기 위함
        # result_df[f'{metric}_distance'] = minmax_scale(result_df[f'{metric}_distance'])
    return result_df

# (angular, euclidean, manhattan, hamming, dot)  

# annoy, 지표, 모듈화

# 지표추출

In [ ]:
# 이전 데이터 load
import pickle
import re
with open("data.pkl", 'rb') as f :
        answer_data = pickle.load(f)
answer_data['contents'] = answer_data['contents'].apply(lambda x: re.sub(r'(<([^>]+)>)', '', x))

### - 지표 확인을 위한 로직 수정 후

In [ ]:
import time
from annoy import AnnoyIndex
import pickle
from sentence_transformers import SentenceTransformer, util
from sklearn.preprocessing import minmax_scale

def start(queries=queries, metric=metric, embedder=embedder, model=model):
    with open("cnt_data.pkl", 'rb') as f:
        cnt_df = pickle.load(f)
    vector_size = 768
    annoy_index = AnnoyIndex(vector_size, metric)
    start_time = time.time()


    queries = [queries]
    
    annoy_index.load('/Users/seunghyun/anaconda3/envs/my_env/23.09.25/ann/' + model + '/topic_classify_' + metric +'.ann')
    
    # 3개만 출력
    KoparentId_list = []
    title_list = []
    distance_list = []
    top_k = 3
    for query in queries:
        query_embedding = embedder.encode(query, convert_to_tensor=True)
            
        top_results = annoy_index.get_nns_by_vector(query_embedding, top_k, include_distances=True)

        end_time = time.time()

        for idx, distance in zip(top_results[0], top_results[1]):
            
            KoparentId_list.append(cnt_df['KoparentId'].iloc[idx])
            title_list.append(cnt_df['title'].iloc[idx])
            distance_list.append(round(distance,4))
        
            
        search_time = end_time - start_time

        
        global result_df
        result_df = pd.DataFrame({'KoparentId': KoparentId_list, "title": title_list, f'{metric}_distance': distance_list})
        # result_df[f'{metric}_distance'] = minmax_scale(result_df[f'{metric}_distance'])
    # return print(cnt_df['KoparentId'].iloc[idx])

### - 두 데이터의 교집합 카테고리를 찾아 평가 지표로 이용

In [ ]:
set(answer_data['title_y'].tolist())
#set(cnt_df['title'].tolist())

### - 평가로직 샘플데이터 추출

In [ ]:
from tqdm import tqdm
import pandas as pd
# 공통 카테고리에 속해있는 문장들을 넣어, 내가만든 모델에서 뽑아준 카테고리 top3와 비교하여 3개중에 하나라도 포함되어 있으면 참으로 지표를 만듬
answer_data_querys = answer_data[(answer_data['title_y'] == '치과') |
                                (answer_data['title_y'] == '의료') | 
                                (answer_data['title_y'] == '교통사고') | 
                                (answer_data['title_y'] == '부동산') |
                                (answer_data['title_y'] == '의료') |
                                (answer_data['title_y'] == '캠핑') |
                                (answer_data['title_y'] == '청소') |
                                (answer_data['title_y'] == '자동차') |
                                (answer_data['title_y'] == '생활꿀팁') |
                                (answer_data['title_y'] == '무역') ]['contents'].tolist()

answer_data_title = answer_data[(answer_data['title_y'] == '치과') |
                                (answer_data['title_y'] == '의료') | 
                                (answer_data['title_y'] == '교통사고') | 
                                (answer_data['title_y'] == '부동산') |
                                (answer_data['title_y'] == '의료') |
                                (answer_data['title_y'] == '캠핑') |
                                (answer_data['title_y'] == '청소') |
                                (answer_data['title_y'] == '자동차') |
                                (answer_data['title_y'] == '생활꿀팁') |
                                (answer_data['title_y'] == '무역') ]['title_y'].tolist()
                                

evaluation_df = pd.DataFrame({'answer_data_querys' : answer_data_querys, 'answer_title' : answer_data_title},columns=['answer_data_querys', 'predict_title', 'answer_title'])
# 클래스별 200개 추출 
sampled_data = pd.concat([group.head(200) for i, group in evaluation_df.groupby('answer_title')])
sampled_data.reset_index(drop=True, inplace=True)
sampled_data['answer_title'].value_counts()

### - 평가할 데이터 프레임 생성

In [ ]:
for idx in tqdm(range(len(sampled_data))):
    
    queries = sampled_data['answer_data_querys'].iloc[idx]
    start(queries,metric,embedder)
    sampled_data.at[idx,'predict_title'] = result_df['title'].tolist()
sampled_data

### - 거리, 모델, 파일 3분류를 for 문을 돌리기위해 리스트, 딕셔너리 형태로 만든다.

In [ ]:
metric_list = ['angular', 'euclidean', 'manhattan', 'hamming', 'dot']

embedder = [SentenceTransformer('jhgan/ko-sroberta-multitask')
         ,SentenceTransformer('smartmind/ko-sbert-augSTS-maxlength512')
         ,SentenceTransformer('intfloat/multilingual-e5-base')
         ,SentenceTransformer('ddobokki/unsup-simcse-klue-roberta-small')
         ,SentenceTransformer('sentence-transformers/facebook-dpr-ctx_encoder-single-nq-base')]

model_name = ['jhgan','smartmind','intfloat','ddobokki','sentence_transformers']

embedder_model_dict = dict(zip(embedder, model_name))

In [ ]:

for embedder, model in embedder_model_dict.items():
    print(model)
    for metric in metric_list:
        print(f'변수명 : {model}_{metric}_sampled_data')
        for idx in tqdm(range(len(sampled_data))):
            queries = sampled_data['answer_data_querys'].iloc[idx]
            start(queries, metric, embedder, model)
            sampled_data.at[idx,'predict_title'] = result_df['title'].tolist()
            sampled_data['Bool'] = False
            for idx in range(len(sampled_data)):
                if sampled_data['answer_title'].iloc[idx] in sampled_data['predict_title'].iloc[idx]:
                sampled_data.at[idx,'Bool'] = True
        globals()[f'{model}_{metric}_sampled_data'] = sampled_data
        

In [ ]:
for i, j in embedder_model_dict.items():

In [ ]:
for value, _ in embedder_dict.items():
    print(_)

In [ ]:
count = []
sampled_data =jhgan_manhattan_sampled_data
sampled_data['Bool'] = False

for idx in range(len(sampled_data)):
    if sampled_data['answer_title'].iloc[idx] in sampled_data['predict_title'].iloc[idx]:
        sampled_data.at[idx,'Bool'] = True

In [ ]:
sampled_data

In [ ]:
round(len(sampled_data[sampled_data['Bool'] == True])/ len(sampled_data),2)

In [ ]:
def start(queries, metric, embedder, model, annoy_path):

In [ ]:
import pandas as pd
# (angular, euclidean, manhattan, hamming, dot)

embedder = SentenceTransformer('jhgan/ko-sroberta-multitask')
# embedder = SentenceTransformer('smartmind/ko-sbert-augSTS-maxlength512')
# embedder = SentenceTransformer('intfloat/multilingual-e5-base')
# embedder = SentenceTransformer('ddobokki/unsup-simcse-klue-roberta-small')
# embedder = SentenceTransformer('sentence-transformers/facebook-dpr-ctx_encoder-single-nq-base')


distance_list = ['angular', 'euclidean', 'manhattan', 'hamming', 'dot']
metric = distance_list[0]


for i in answer_data[answer_data['title_y'] == '부동산']['contents']:
    queries = i
    start(queries,metric,embedder)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

value = angular_df['angular_distance']
x = np.array(value)
plt.figure(figsize=(5,5))
plt.title('angular_distance')
plt.xlabel('Distance')
plt.ylabel('빈도')
plt.grid()
plt.plot(x, norm.pdf(x, loc=sum(value)/len(value), scale=0.5))
plt.show()


---

In [ ]:
from transformers import pipeline

pipe = pipeline(model="facebook/bartlargemnli")

In [ ]:
start_time = time.time()
sequence = "질문 편의점에 흔히 볼수있는 달달한 간식추천좀요"	
category_label = df['title'].values.tolist()
output = pipe(sequence,
    category_label
)
print(f"입력 문장: {sequence}")
print("KoparentId :", mrg_df[mrg_df['title'] == output['labels'][output['scores'].index(max(output['scores']))]]['KoparentId'].values[0])
print("parentId :", mrg_df[mrg_df['title'] == output['labels'][output['scores'].index(max(output['scores']))]]['parentId'].values[0])
print('title :', output['labels'][output['scores'].index(max(output['scores']))])
print("확률 :",round(output['scores'][output['scores'].index(max(output['scores']))], 4))

end_time = time.time()

search_time = end_time  start_time
print(f"검색 시간: {search_time:.4f} 초")

### * sbert를 이용하여 topic classify
* jhgan/kosrobertamultitask
* intfloat/multilinguale5small

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import time

# embedder = SentenceTransformer("jhgan/kosrobertamultitask")
# embedder = SentenceTransformer('intfloat/multilinguale5small')
# embedder = SentenceTransformer('smartmind/kosbertaugSTSmaxlength512')
embedder = SentenceTransformer('intfloat/multilinguale5base')

sentences = mrg_df['re_title'].tolist()
embedding_start = time.time()
# 문장을 tensor 형태로 변환
corpus_embeddings = embedder.encode(sentences, convert_to_tensor=True)
embedding_end = time.time()
embedding_time = embedding_end  embedding_start
print(f"임베딩 시간 {embedding_time} 시간")      

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import time

# embedder = SentenceTransformer("jhgan/kosrobertamultitask")
# embedder = SentenceTransformer('intfloat/multilinguale5small')
# embedder = SentenceTransformer('smartmind/kosbertaugSTSmaxlength512')
# embedder = SentenceTransformer('intfloat/multilinguale5base')
sentences = mrg_df['re_title'].tolist()
embedding_start = time.time()
# 문장을 tensor 형태로 변환
corpus_embeddings = embedder.encode(sentences, convert_to_tensor=True)
embedding_end = time.time()
embedding_time = embedding_end  embedding_start
print(f"임베딩 시간 {embedding_time/60/60} 시간")      


# sroberta
from sentence_transformers import SentenceTransformer, util
import pickle
import numpy as np

# # 문장 불러오기
# sentences = mrg_df['re_title'].tolist()
# 검색
import time
queries = """이혼시 상속재산 분할 받으려면 딱 10년 채워야 하나요"""
queries = """아나바다운동이 뭔가요?"""
queries = [queries]
embedder = SentenceTransformer("jhgan/kosrobertamultitask")
# embedder = SentenceTransformer('intfloat/multilinguale5small')
# embedder = SentenceTransformer('smartmind/kosbertaugSTSmaxlength512')
# embedder = SentenceTransformer('intfloat/multilinguale5base')
# embedder = SentenceTransformer('ddobokki/unsupsimcsekluerobertasmall')
# embedder = SentenceTransformer('sentencetransformers/facebookdprctx_encodersinglenqbase')

# 3개만 출력
top_k = 230
sum_cos_scores = 0
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    
    start_time = time.time()
    # input 문장과, re_title문장들의 embedding값을 코사인 유사도로 비교
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    #We use np.argpartition, to only partially sort the top_k results
    # 순위를 순서대로 맞추기 위함
    # np.argpartition는 큰값 5개를 뽑아 오른쪽으로 넣는 함수
    top_results = np.argpartition(cos_scores, range(top_k))[0:top_k] 
    
    end_time = time.time()

    print("\n======================\n")
    print("검색어: ", query)
    print(f"검색결과 {top_k}개 출력")
    print("\n======================\n")
    # score가 높은 top5개를 뽑고 score도 부여한다.
    for index, idx in enumerate(top_results[0:top_k]):
        print(f"{index+1}번쨰 :","중분류 :",mrg_df['KoparentId'].iloc[int(idx)].strip(),"| 소분류 :", mrg_df['title'].iloc[int(idx)].strip(), "(Score: %.4f)" % (cos_scores[idx]))
        sum_cos_scores += cos_scores[idx]
        
    print(" ")
    search_time = end_time - start_time
    print(f"검색 시간: {search_time:.4f} 초")
    
print(f"코사인 스코어 합: {round(float(sum_cos_scores/top_k),4)}")

# 10/6(금) 확인부분 model 별로 출력값의 코사인 유사도가 얼마나 분포해 있는지 확인하였음.
# jhgan/kosrobertamultitask 코사인 범위 0.42 ~ 0.03
# intfloat/multilinguale5base 코사인 스코어 : 0.83 ~ 0.77
# 모델을 더 다양하게 써보았는데 좋은 결과 x / 문장을 고쳐보았는데
# 기존데이터는 sbert에서 아하앤 컴퍼니에 있는 데이터셋으로 파인튜닝으로 학습을 진행 > 
# 중의적은 문장은 어떻게 하느냐?
# 새로운 topic 데이터셋은 > topic classify로 만들고 모델을 선정
# sbert에 아하엔 사이트의 카테고리  문장의 유사도를 학습
# 어떤 문장은 이게더 잘나오고 어떤문장은 다른 문장이 더 잘나옴. > 

# annoy, 지표, 모듈화

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import time

# embedder = SentenceTransformer("jhgan/kosrobertamultitask")
embedder = SentenceTransformer('intfloat/multilinguale5small')
# embedder = SentenceTransformer('smartmind/kosbertaugSTSmaxlength512')
# embedder = SentenceTransformer('intfloat/multilinguale5base')
sentences = mrg_df['re_title'].tolist()
embedding_start = time.time()
# 문장을 tensor 형태로 변환
corpus_embeddings = embedder.encode(sentences, convert_to_tensor=True)
embedding_end = time.time()
embedding_time = embedding_end  embedding_start
print(f"임베딩 시간 {embedding_time/60/60} 시간")      

In [ ]:
Compared_df.T

In [ ]:
import pandas as pd
import numpy as np
Compared_df = pd.DataFrame(np.arange(0,16).reshape(4,4),
                    columns = ["jhgan/kosrobertamultitask",
                                'intfloat/multilinguale5small',
                                'smartmind/kosbertaugSTSmaxlength512',
                                'intfloat/multilinguale5base'])
Compared_df.index = ['Input_Text','중분류','소분류','코사인 유사도']

In [ ]:
# Annoy metric 별 유사도 합 추출 
# 다른문장을 10회 정도 넣어서 유사도를 측정해보았는데 평균값이 0번모델 1.0~1.2, 2.5~2.6으로 측정됨.
# jhgan/kosrobertamultitask > intfloat/multilinguale5small cosine score는 낮지만, 범위가 넓게 분포되어 있어 3개까지는 더 정확하다고 볼수 있는가?
# 여러개를 출력하면 더 맞는 카테고리 근접할수 있다.
evl = {'model' : ['jhgan/kosrobertamultitask', 'intfloat/multilinguale5small'], "cosine_scores" : [1.1971/3, 2.503/3]}
model_evl = pd.DataFrame(evl)
model_evl.T

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import time

embedder = SentenceTransformer("jhgan/kosrobertamultitask")
# embedder = SentenceTransformer('intfloat/multilinguale5small')
# embedder = SentenceTransformer('smartmind/kosbertaugSTSmaxlength512')
# embedder = SentenceTransformer('intfloat/multilinguale5base')
sentences = mrg_df['re_title'].tolist()
embedding_start = time.time()
# 문장을 tensor 형태로 변환
corpus_embeddings = embedder.encode(sentences, convert_to_tensor=True)
embedding_end = time.time()
embedding_time = embedding_end  embedding_start
print(f"임베딩 시간 {embedding_time/60/60} 시간")      

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import time

# embedder = SentenceTransformer("jhgan/kosrobertamultitask")
embedder = SentenceTransformer('intfloat/multilinguale5small')
sentences = mrg_df['re_title'].tolist()
embedding_start = time.time()
# 문장을 tensor 형태로 변환
corpus_embeddings = embedder.encode(sentences, convert_to_tensor=True)
embedding_end = time.time()
embedding_time = embedding_end  embedding_start
print(f"임베딩 시간 {embedding_time/60/60} 시간")
      